In [ ]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import billboard

#import applemusicpy
import lyricsgenius
#import SpotifyCharts as sCharts
import csv
import os
from textstat.textstat import textstat
import threading as thread
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np
from sklearn import linear_model, mixture
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [3]:
path = "C:/Users/Chaitu Konjeti/SongPopularityPredictionAlgorithm/output"
genius = lyricsgenius.Genius("Xf0XfBJTZon0Sra2rGV56TAXp6jOUaLJVhmHxqbTW5mp-j6S2NVcmHWSLQ29v0dk")

In [4]:
try:
    os.mkdir(path)
except:
    pass

In [5]:
def writeSongCharacteristics(i, billboardChart, songs):
    key = str(billboardChart[i].title + billboardChart[i].artist)

    if key not in songs:
        song = billboardChart[i]
        title = billboardChart[i].title
        artist = song.artist
        peakpos = song.peakPos
        lastpos = song.lastPos
        numWeeks = song.weeks
        currentPos = song.rank
        isNew = song.isNew
        lyrics = genius.search_song(title, artist).lyrics
        row = [title, artist, lyrics, peakpos, lastpos, numWeeks, currentPos, isNew]
        songs[key] = row

    if key in songs:
        if billboardChart[i].peakPos != songs.get(key)[3]:
            newRow = songs.get(key)
            newRow[3] = billboardChart[i].peakPos
            songs[key] = newRow
        if billboardChart[i].weeks != songs.get(key)[5]:
            newRow = songs.get(key)
            newRow[5] = billboardChart[i].weeks
            songs[key] = newRow

In [6]:
def getAllSongData(startMonth, startYear, endMonth, endYear, numSongs):
    for year in range(startYear, endYear + 1):
        thread.Thread(None, target=yearlySongData, args=(year, startMonth, endMonth, numSongs)).start()

In [7]:
def yearlySongData(year, startMonth, endMonth, numSongs):
    for month in range (startMonth, endMonth + 1):
            thread.Thread(None, target=monthlySongData, args=(year, month, numSongs)).start()

In [9]:
def monthlySongData(year, month, numSongs):
        outputFileName = "C:/Users/Chaitu Konjeti/SongPopularityPredictionAlgorithm/output/billboardHot100_Lyrics_{}_{}.csv".format(year, month)

        with open(outputFileName, 'a+', newline='', encoding='utf-8') as outputFile:
            songs = {}
            day = 1
            dataWriter = csv.writer(outputFile)

            while (day <= 31):
                if month in range(1, 10):
                    try:
                        billboardChart = billboard.ChartData('hot-100', date = "{}-{:02d}-{}".format(year, month, day))
                        for i in range(0, numSongs):
                            try:
                                writeSongCharacteristics(i, billboardChart, songs)
                            except:
                                pass
                    except:
                        pass

                elif month in range(10, 13):
                    try:
                        billboardChart = billboard.ChartData('hot-100', date = "{}-{}-{}".format(year, month, day))
                        for i in range(0, numSongs):
                            try:
                                writeSongCharacteristics(i, billboardChart, songs)
                            except:
                                pass
                    except:
                        pass
                day += 5

            for key in songs:
                if (len(songs[key][2]) < 10000):
                    dataWriter.writerow(songs[key])

In [10]:
def lyric_preprocessing(dir):
    for filename in os.listdir(dir):
        data = []
        sid = SentimentIntensityAnalyzer()
        with open(dir + filename, encoding='utf-8', mode='r+') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            for row in csv_reader:
                temp = row
                num_positive = 0
                num_negative = 0
                num_neutral = 0
                dale_chall_score = 0
                #print(row[0])
                lyrics = row[2]
                lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
                lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])
                #print(lyrics)
                #this_sentence = lyrics.decode('utf-8')
                #numlines = 0
                for line in lyrics.splitlines():
                    #print(line)
                    comp = sid.polarity_scores(line)
                    #print(comp)
                    comp = comp['compound']
                    if comp >= 0.5:
                        num_positive += 1
                    elif comp > -0.5 and comp < 0.5:
                        num_neutral += 1
                    else:
                        num_negative += 1

                    #numlines += 1

                temp.append(num_positive)
                temp.append(num_neutral)
                temp.append(num_negative)
                temp.append(textstat.dale_chall_readability_score(lyrics))
                temp.append(textstat.difficult_words(lyrics))
                data.append(temp)
                #print(temp)
            outputFileName = "C:/Users/Chaitu Konjeti/song-popularity-prediction-algorithm/processed/" + filename

            with open(outputFileName, 'a+', newline='', encoding='utf-8') as outputFile:
                dataWriter = csv.writer(outputFile)
                for items in data:
                    #print(items)
                    dataWriter.writerow(items)

In [ ]:
getAllSongData(1, 2000, 12, 2019, 100)

In [ ]:
lyric_preprocessing("output/")

In [15]:
X = []
y = []

In [16]:
dir = 'C:/Users/Chaitu Konjeti/song-popularity-prediction-algorithm/processed/'

In [17]:
for filename in os.listdir(dir):
    with open(dir + filename, encoding='utf-8-sig', mode='r') as csv_file:
        df = pd.read_csv(dir + filename, header = None)
        #print(df.head())
        df.columns = ['title', 'artist', 'lyrics', 'peakpos', 'lastpos', 'numWeeks', 'currentPos', 'isNew', 'pos', 'neu', 'neg', 'dale', 'diff']
        # #print(df['title'])
        # features = list(df.columns.values)
        # features.remove('title')
        # features.remove('artist')
        # features.remove('currentPos')
        # features.remove('lyrics')
        # X.append(df[features])
        # y.append(df['currentPos'])
        for lyric in df['lyrics']:
            X.append(lyric)
        for pos in df['currentPos']:
            y.append(pos)

In [18]:
count_vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 3))
X = count_vectorizer.fit_transform(X)
y = np.asarray(y)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [20]:
clf = LogisticRegression(verbose=1)
clf.fit(X_train, y_train)

C:\Users\Chaitu Konjeti\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Chaitu Konjeti\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[LibLinear]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=1,
                   warm_start=False)

In [21]:
clf.score(X_train, y_train)

1.0

In [22]:
pred = clf.predict(X_test)

In [23]:
pred

array([49, 97, 29, 52, 29, 16, 97,  8, 97, 97, 99, 47, 97, 52, 97, 67, 97,
       33, 97, 52, 67, 97, 99, 82, 26, 10, 85])

In [24]:
y_test

array([43, 37, 94,  6, 66, 98, 57, 18, 54, 83, 70, 79, 45, 60, 90, 62, 34,
       79, 31, 17, 49, 93, 46, 32,  5, 40, 78])